<a href="https://colab.research.google.com/github/Ao-Baid/steamrecommendation/blob/main/Steam_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import necessary libraries for data visualisation, training and evaluation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import csv

Loading the data

In [ ]:
root_dir = '/content/drive/MyDrive/Steam Recommendations'

df_games = pd.read_csv(root_dir + '/games.csv')
df_games_meta = pd.read_json(root_dir + '/games_metadata.json', lines=True, orient="records")
df_recommendations = pd.read_csv(root_dir + '/recommendations.csv')
df_users = pd.read_csv(root_dir + '/users.csv')

pd.DataFrame([
    ["Recommendations", len(df_recommendations.index)],
    ["Users", len(df_users.index)],
    ["Games", len(df_games.index)],
    ["Games Metadata", len(df_games_meta.index)]
], columns=["Dataframe", "Records"])

,Dataframe,Records
0,Recommendations,41154794
1,Users,14306064
2,Games,50872
3,Games Metadata,50872


A clearer view of the games tables and the attributes it contains:


*   app_id (native product ID on steam)
*   title
*   date_release
*   OS support
*   Product rating type
*   Ratio of positive feedbacks
*   Number of reviews
*   Pricing (original and discounted)
*   Steam Deck compatibility




In [ ]:
df_games.head()

,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck
0,13500,Prince of Persia: Warrior Within™,2008-11-21,True,False,False,Very Positive,84,2199,9.99,9.99,0.0,True
1,22364,BRINK: Agents of Change,2011-08-03,True,False,False,Positive,85,21,2.99,2.99,0.0,True
2,113020,Monaco: What's Yours Is Mine,2013-04-24,True,True,True,Very Positive,92,3722,14.99,14.99,0.0,True
3,226560,Escape Dead Island,2014-11-18,True,False,False,Mixed,61,873,14.99,14.99,0.0,True
4,249050,Dungeon of the ENDLESS™,2014-10-27,True,True,False,Very Positive,88,8784,11.99,11.99,0.0,True


Users table contains the attributes:


*   user_id (Primary key)
*   Number of products purchased by the user
*   Number of reviews published on steam



In [ ]:
df_users.head()